# Creating Attributes at Run-Time

## We already saw that we can add data attributes to instances at run-time, and that it affects just that single instance:

In [1]:
class Person:
    def __init__(self):
        pass
    
    def foothill(self):
        print("Foothill")

In [2]:
p1 = Person()
p2 = Person()

print(p1.__dict__)
print(p2.__dict__)

{}
{}


In [3]:
p1.foothill()
p2.foothill()

Foothill
Foothill


In [4]:
p1.name = 'Alex'

In [5]:
p1.__dict__

{'name': 'Alex'}

In [7]:
p2.__dict__

{}

### So what happens if we add a function as an attribute to our instances directly (we can even do the same within an `__init__` method, it works the same way)   
- Remember that if we add a function to the class itself, calling the function from the instance will result in it being treated as a method. 
- Here, the result is different, since we are adding the function directly to the instance, not the class:

In [8]:
def say_hello():
    return "Hello"

p1.say_hello = say_hello

In [9]:
p1.__dict__

{'name': 'Alex', 'say_hello': <function __main__.say_hello()>}

### As you can see, that attribute is a **plain** function - it is **not** being interpreted as a **method**.

In [14]:
p1.say_hello()

'Hello'

Of course, the other instances do not know anything about that function:

In [15]:
p2.__dict__

{}

### So, the question becomes, **can** we create a **method** on a specific instance?    
- The answer (of course!) is yes, but we have to explicitly tell Python we are setting up a method bound to that specific instance.    
- We do this by creating a `method` type object:

In [10]:
from types import MethodType

In [11]:
class Person:
    def __init__(self, name):
        self.name = name

In [12]:
p1 = Person('Eric')
p2 = Person('Alex')

### Now let's create a `method` object, and bind it to `p1`.     
- First we create a function that will handle the bound object as it's first argument, and use the instance `name` property.

In [13]:
def say_hello(self):
    return f'{self.name} says hello!'

### Now we can use that function just by itself, passing in any object that has a `name` attribute:

In [14]:
say_hello(p1), say_hello(p2)

('Eric says hello!', 'Alex says hello!')

### Notice that we passed the objects p1 and p2 to the function

### How does this work?  
- This is just a normal function

### Now however, we are going to create a method bound to `p1` 

#### specifically:    
- The function MethodType accepts 2 arguments    
 - say_hello is the function being bound as a method
 - p1 is an instance (object) that we are binding it to

In [21]:
def say_hello(self):
    return f'{self.name} says hello!'

p1_say_hello = MethodType(say_hello, p1)

In [22]:
p1_say_hello

<bound method say_hello of <__main__.Person object at 0x0000019BC0A6E3D0>>

In [23]:
f"id of p1: {hex(id(p1))}, id of p2: {hex(id(p2))}"

'id of p1: 0x19bc0a6e3d0, id of p2: 0x19bc0a6e340'

As you can see that method is bound to the instance `p1`. But how do we call it?

If we try to use dotted notation or a `getattr`, this will not work because the `p1` object does not know anything about that method:

In [24]:
try:
    p1.p1_say_hello()
except AttributeError as ex:
    print(ex)

'Person' object has no attribute 'p1_say_hello'


All we need to do is add that method to the instance dictionary - giving it whatever name we want:

In [25]:
p1.say_hello = p1_say_hello   

In [26]:
p1.__dict__

{'name': 'Eric',
 'say_hello': <bound method say_hello of <__main__.Person object at 0x0000019BC0A6E3D0>>}

OK, so now our instance knows about the method that we stored under the name `say_hello`:

In [27]:
p1.say_hello()

'Eric says hello!'

or, we can use the `getattr` function:

In [28]:
getattr(p1, 'say_hello')()

'Eric says hello!'

And of course, othe instances know nothing about this:

In [29]:
p2.__dict__

{'name': 'Alex'}

So, to create a bound method after the object has initially been created, we just create a bound method and add it to the instance itself.

We can do it this way (what we just saw):

In [30]:
p1 = Person('Alex')
p1.__dict__

{'name': 'Alex'}

MethodType is a class which converts a function to a method   
`MethodType(func, instance)`    
- It actually takes a function object, converts it to a method and returns the id of the new method 

In [31]:
p1.say_hello = MethodType(lambda self: f'{self.name} says hello', p1)

### Here we first create the lambda function named say_hello    
- Then we bind it to p1

In [32]:
f"{type(say_hello)} id: {hex(id(say_hello))}"   # say hello is a function

"<class 'function'> id: 0x19bc0b27820"

In [33]:
f"{type(p1.say_hello)} id: {hex(id(p1.say_hello))}" # p1.say_hello() is a bound method

"<class 'method'> id: 0x19bc0a1f700"

In [34]:
p1.say_hello()

'Alex says hello'

### But we can also do this from any instance method.

#### Example

- Suppose we want some class to have some functionality that is called the same way but will differ from instance to instance.    
 - Although we could use inheritance, here I want some kind of 'plug-in' approach and we can do this without inheritance, mixins, or anything like that! (to be covered is C S 3B)

In [35]:
from types import MethodType

class Person:
    def __init__(self, name):
        self.name = name
        
    def register_do_work(self, func):
        setattr(self, '_do_work', MethodType(func, self))
        
    def do_work(self):
        do_work_method = getattr(self, '_do_work', None)
        # if attribute exists we'll get it back, otherwise it will be None
        if do_work_method:
            return do_work_method()
        else:
            raise AttributeError('You must first register a do_work method')

In [36]:
math_teacher = Person('Alice')
english_teacher = Person('John')

Right now neither the math nor the english teacher can do any work because we haven't "registered" a worker yet:  
* An aside, I want to demonstrate another way to generate a message in a try/except block

In [37]:
math_teacher.do_work()

AttributeError: You must first register a do_work method

In [38]:
try:
    math_teacher.do_work()
except AttributeError as ex:
    print(ex)

You must first register a do_work method


### Ok, so let's do that:

In [39]:
def work_math(self):
     return f'{self.name} will teach differentials today.'

#### Call math_teacher.register_do_work(work_math)     this passes the work_math function object

In [40]:
math_teacher.register_do_work(work_math)

In [41]:
math_teacher.__dict__

{'name': 'Alice',
 '_do_work': <bound method work_math of <__main__.Person object at 0x0000019BC0A6ED30>>}

In [42]:
math_teacher.do_work()

'Alice will teach differentials today.'

And we can create a different `do_work` method for the English teacher:

In [53]:
def work_english(self):
    return f'{self.name} will analyze Hamlet today.'

In [54]:
english_teacher.register_do_work(work_english)

In [55]:
english_teacher.do_work()

'John will analyze Hamlet today.'

#### Keep reviewing this example until it makes sense to you